# loading the data from h5py is incredibly slow, how can we 

In [ ]:
#so we installed the tfrecord package, let's use that

#here's the original code

'''
import json
import functools
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import numpy as np
import h5py
import tensorflow as tf
import sonnet as snt
from IPython.display import clear_output
import time
import sys
sys.path.append('/data/leslie/sarthak/data/enformer')
import enformer


def organism_path(organism):
  return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism) #just leads to the organism path, quite easy


def get_dataset(organism, subset, num_threads=4): #this gets the data and deserializes it
  metadata = get_metadata(organism)
  dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset


def get_metadata(organism):
  # Keys:
  # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
  # pool_width, crop_bp, target_length
  path = os.path.join(organism_path(organism), 'statistics.json')
  with tf.io.gfile.GFile(path, 'r') as f:
    return json.load(f)


def tfrecord_files(organism, subset): #this is the function that gets the TFR files, gets all of them and sorts them
  # Sort the values by int(*).
  return sorted(tf.io.gfile.glob(os.path.join(
      organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
  )), key=lambda x: int(x.split('-')[-1].split('.')[0]))


def deserialize(serialized_example, metadata):
  """Deserialize bytes stored in TFRecordFile."""
  feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
  }
  example = tf.io.parse_example(serialized_example, feature_map)
  sequence = tf.io.decode_raw(example['sequence'], tf.bool)
  sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
  sequence = tf.cast(sequence, tf.float32)

  target = tf.io.decode_raw(example['target'], tf.float16)
  target = tf.reshape(target,
                      (metadata['target_length'], metadata['num_targets']))
  target = tf.cast(target, tf.float32)

  return {'sequence': sequence,
          'target': target}

def get_targets(organism):
  targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
  return pd.read_csv(targets_txt, sep='\t')
df_targets_human = get_targets('human')
# df_targets_human.head()

human_dataset = get_dataset('human', 'train')
'''

In [2]:
#what we want to do is load it in with minimal changes using tfrecord
import os
import json
import functools
import pandas as pd
import glob
import numpy as np
from tfrecord.torch.dataset import TFRecordDataset
# from tfrecord.torch.dataset import load_schema

def organism_path(organism):
    return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism)

def get_metadata(organism):
    path = os.path.join(organism_path(organism), 'statistics.json')
    with open(path, 'r') as f:
        return json.load(f)

def tfrecord_files(organism, subset):
    return sorted(glob.glob(os.path.join(
        organism_path(organism), 'tfrecords', f'{subset}-*.tfrecord'
    )), key=lambda x: int(x.split('-')[-1].split('.')[0]))

def parse_tfrecord(serialized_example, metadata):
    feature_map = {
        'sequence': 'byte',
        'target': 'byte',
    }
    example = TFRecordDataset.deserialize_example(serialized_example, feature_map)
    
    sequence = np.frombuffer(example['sequence'], dtype=np.bool)
    sequence = sequence.reshape((metadata['seq_length'], 4))
    sequence = sequence.astype(np.float32)

    target = np.frombuffer(example['target'], dtype=np.float16)
    target = target.reshape((metadata['target_length'], metadata['num_targets']))
    target = target.astype(np.float32)

    return {'sequence': sequence, 'target': target}

def get_dataset(organism, subset, num_threads=4):
    metadata = get_metadata(organism)
    files = tfrecord_files(organism, subset)
    
    dataset = TFRecordDataset(
        files,
        index_path=None,
        description=None,
        transform=functools.partial(parse_tfrecord, metadata=metadata),
        shuffle_queue_size=2048
    )
    
    return dataset

def get_targets(organism):
    targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
    return pd.read_csv(targets_txt, sep='\t')

df_targets_human = get_targets('human')
human_dataset = get_dataset('human', 'train')

In [9]:
#wait it was that simple?
#now iterate
human_eval_dataset = get_dataset('human', 'valid')

In [10]:
human_eval_dataset

In [11]:
for i, data in enumerate(human_dataset):
    sequence = data['sequence']
    target = data['target']
    print(f"Sample {i}:")
    print(f"Sequence shape: {sequence.shape}")
    print(f"Target shape: {target.shape}")
    
    # Just accessing the first few samples for demonstration
    if i == 2:
        break

TypeError: expected str, bytes or os.PathLike object, not list

In [13]:
from torch.utils.data import DataLoader

# Create a DataLoader for the dataset
dataloader = DataLoader(human_dataset, batch_size=1)

# Iterate over the DataLoader
for batch in dataloader:
    sequences = batch['sequence']
    targets = batch['target']
    print(f"Batch sequences shape: {sequences.shape}")
    print(f"Batch targets shape: {targets.shape}")
    
    # Accessing only the first batch for demonstration
    break

TypeError: expected str, bytes or os.PathLike object, not list

In [14]:
#I'm going to assume chatgpt just made it completely wrong
def organism_path(organism):
  return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism) #just leads to the organism path, quite easy


def get_dataset(organism, subset, num_threads=4): #this gets the data and deserializes it
  metadata = get_metadata(organism)
  dataset = TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset


# def get_metadata(organism):
#   # Keys:
#   # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
#   # pool_width, crop_bp, target_length
#   path = os.path.join(organism_path(organism), 'statistics.json')
#   with tf.io.gfile.GFile(path, 'r') as f:
#     return json.load(f)
  
def get_metadata(organism):
    path = os.path.join(organism_path(organism), 'statistics.json')
    with open(path, 'r') as f:
        return json.load(f)

def tfrecord_files(organism, subset):
    return sorted(glob.glob(os.path.join(
        organism_path(organism), 'tfrecords', f'{subset}-*.tfrecord'
    )), key=lambda x: int(x.split('-')[-1].split('.')[0]))
    
def parse_tfrecord(serialized_example, metadata):
    feature_map = {
        'sequence': 'byte',
        'target': 'byte',
    }
    example = TFRecordDataset.deserialize_example(serialized_example, feature_map)
    
    sequence = np.frombuffer(example['sequence'], dtype=np.bool_)
    sequence = sequence.reshape((metadata['seq_length'], 4))
    sequence = sequence.astype(np.float32)

    target = np.frombuffer(example['target'], dtype=np.float16)
    target = target.reshape((metadata['target_length'], metadata['num_targets']))
    target = target.astype(np.float32)

    return {'sequence': sequence, 'target': target}

def get_dataset(organism, subset, num_threads=4):
    metadata = get_metadata(organism)
    files = tfrecord_files(organism, subset)
    
    dataset = TFRecordDataset(
        files,
        index_path=None,
        description=None,
        transform=functools.partial(parse_tfrecord, metadata=metadata),
        shuffle_queue_size=2048
    )
    
    return dataset

def get_targets(organism):
    targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
    return pd.read_csv(targets_txt, sep='\t')

# Load the metadata and dataset
df_targets_human = get_targets('human')
human_dataset = get_dataset('human', 'train')

# Iterate over the dataset and access the data
for i, data in enumerate(human_dataset):
    sequence = data['sequence']
    target = data['target']
    print(f"Sample {i}:")
    print(f"Sequence shape: {sequence.shape}")
    print(f"Target shape: {target.shape}")
    
    # Just accessing the first few samples for demonstration
    if i == 2:
        break

TypeError: expected str, bytes or os.PathLike object, not list

In [15]:
#ok let's do things one at a time, we have som ekey functions we assume work just fine

#get dataset requires deserialize
#we have an alternate to get_metadata

def get_metadata(organism):
    path = os.path.join(organism_path(organism), 'statistics.json')
    with open(path, 'r') as f:
        return json.load(f)
metadata = get_metadata('human')
metadata
#this works just fine

{'num_targets': 5313,
 'train_seqs': 34021,
 'valid_seqs': 2213,
 'test_seqs': 1937,
 'seq_length': 131072,
 'pool_width': 128,
 'crop_bp': 8192,
 'target_length': 896}

In [16]:
def tfrecord_files(organism, subset):
    return sorted(glob.glob(os.path.join(
        organism_path(organism), 'tfrecords', f'{subset}-*.tfrecord'
    )), key=lambda x: int(x.split('-')[-1].split('.')[0]))

files = tfrecord_files('human', 'train')
files
#no so this seems to have an issue

[]

In [17]:
organism_path('human')

'/data/leslie/sarthak/data/enformer/data/human'

In [18]:
def organism_path(organism):
    return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism)

def tfrecord_files(organism, subset):
    # Use glob to get all matching files
    file_pattern = os.path.join(organism_path(organism), 'tfrecords', f'{subset}-*.tfr')
    file_list = glob.glob(file_pattern)
    
    # Sort the files based on the integer value in the filename
    sorted_files = sorted(file_list, key=lambda x: int(x.split('-')[-1].split('.')[0]))
    
    return sorted_files

# Example usage:
organism = 'human'
subset = 'train'
files = tfrecord_files(organism, subset)
print(files)

['/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-0.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-1.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-2.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-3.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-4.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-5.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-6.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-7.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-8.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-9.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-10.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-11.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-12.tfr', '/data/leslie/sarthak/data/enformer/data/human/

In [19]:
#see this works!

def parse_tfrecord(serialized_example, metadata):
    feature_map = {
        'sequence': 'byte',
        'target': 'byte',
    }
    example = TFRecordDataset.deserialize_example(serialized_example, feature_map)
    
    sequence = np.frombuffer(example['sequence'], dtype=np.bool_)
    sequence = sequence.reshape((metadata['seq_length'], 4))
    sequence = sequence.astype(np.float32)

    target = np.frombuffer(example['target'], dtype=np.float16)
    target = target.reshape((metadata['target_length'], metadata['num_targets']))
    target = target.astype(np.float32)

    return {'sequence': sequence, 'target': target}

def get_dataset(organism, subset, num_threads=4):
    metadata = get_metadata(organism)
    files = tfrecord_files(organism, subset)
    
    dataset = TFRecordDataset(
        files,
        index_path=None,
        description=None,
        transform=functools.partial(parse_tfrecord, metadata=metadata),
        shuffle_queue_size=2048
    )
    
    return dataset

dataset = get_dataset('human', 'train')

In [20]:
next(iter(dataset))

TypeError: expected str, bytes or os.PathLike object, not list

In [25]:
#so the issue is not there, we now can give it the right list

import numpy as np
import struct

def deserialize(serialized_example, metadata):
    """Deserialize bytes stored in TFRecordFile."""
    # Define feature map structure
    feature_map = {
        'sequence': ('sequence', np.float32),
        'target': ('target', np.float32),
    }

    # Extract feature values
    sequence_str_len = metadata['seq_length'] * 4
    target_str_len = metadata['target_length'] * metadata['num_targets'] * 2

    # Split serialized_example into sequence and target parts
    sequence_bytes = serialized_example[:sequence_str_len]
    target_bytes = serialized_example[sequence_str_len:sequence_str_len + target_str_len]

    # Decode the sequence and target using numpy
    sequence = np.frombuffer(sequence_bytes, dtype=np.uint8).astype(np.bool_)
    sequence = np.reshape(sequence, (metadata['seq_length'], 4)).astype(np.float32)

    target = np.frombuffer(target_bytes, dtype=np.float16).astype(np.float32)
    target = np.reshape(target, (metadata['target_length'], metadata['num_targets']))

    return {'sequence': sequence, 'target': target}



In [22]:
human_dataset = get_dataset('human', 'train')

In [23]:
next(iter(human_dataset))

TypeError: expected str, bytes or os.PathLike object, not list

In [26]:
#what if we just gave it a single example
def get_dataset(organism, subset, num_threads=4):
    metadata = get_metadata(organism)
    files = tfrecord_files(organism, subset)
    
    dataset = TFRecordDataset(
        files[0],
        index_path=None,
        description=None,
        transform=functools.partial(parse_tfrecord, metadata=metadata),
        shuffle_queue_size=2048
    )
    
    return dataset

human_dataset = get_dataset('human', 'train')
print(next(iter(human_dataset)))

OverflowError: Python int too large to convert to C ssize_t

# ok take a few steps back, what are we doing

In [28]:
#what we know for sure is this
def organism_path(organism):
    return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism)

def tfrecord_files(organism, subset):
    # Use glob to get all matching files
    file_pattern = os.path.join(organism_path(organism), 'tfrecords', f'{subset}-*.tfr')
    file_list = glob.glob(file_pattern)
    
    # Sort the files based on the integer value in the filename
    sorted_files = sorted(file_list, key=lambda x: int(x.split('-')[-1].split('.')[0]))
    
    return sorted_files

def get_targets(organism):
  targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
  return pd.read_csv(targets_txt, sep='\t')

def get_metadata(organism):
    path = os.path.join(organism_path(organism), 'statistics.json')
    with open(path, 'r') as f:
        return json.load(f)

In [ ]:
#what we need to fix is 
def get_dataset(organism, subset, num_threads=4): #this gets the data and deserializes it
  metadata = get_metadata(organism)
  dataset = TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset

#but what we need is deserialize to work

In [34]:
#first get a dataset from the TFRecordDataset
metadata = get_metadata('human')
dataset = TFRecordDataset(tfrecord_files('human', 'train'), None)
#wait seems to only accept gzip or None...

In [32]:
#also it doesn't support zlib compression, so have to manually do it

In [ ]:
# def deserialize(serialized_example, metadata):
#   """Deserialize bytes stored in TFRecordFile."""
#   feature_map = {
#       'sequence': tf.io.FixedLenFeature([], tf.string),
#       'target': tf.io.FixedLenFeature([], tf.string),
#   }
#   example = tf.io.parse_example(serialized_example, feature_map)
#   sequence = tf.io.decode_raw(example['sequence'], tf.bool)
#   sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
#   sequence = tf.cast(sequence, tf.float32)

#   target = tf.io.decode_raw(example['target'], tf.float16)
#   target = tf.reshape(target,
#                       (metadata['target_length'], metadata['num_targets']))
#   target = tf.cast(target, tf.float32)

#   return {'sequence': sequence,
#           'target': target}

# just installed tensorflow let's try this

In [2]:
import json
import functools
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import numpy as np
import h5py
import tensorflow as tf
# import sonnet as snt
from IPython.display import clear_output
import time
import sys
sys.path.append('/data/leslie/sarthak/data/enformer')
# import enformer


def organism_path(organism):
  return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism) #just leads to the organism path, quite easy


def get_dataset(organism, subset, num_threads=4): #this gets the data and deserializes it
  metadata = get_metadata(organism)
  dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset


def get_metadata(organism):
  # Keys:
  # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
  # pool_width, crop_bp, target_length
  path = os.path.join(organism_path(organism), 'statistics.json')
  with tf.io.gfile.GFile(path, 'r') as f:
    return json.load(f)


def tfrecord_files(organism, subset): #this is the function that gets the TFR files, gets all of them and sorts them
  # Sort the values by int(*).
  return sorted(tf.io.gfile.glob(os.path.join(
      organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
  )), key=lambda x: int(x.split('-')[-1].split('.')[0]))


def deserialize(serialized_example, metadata):
  """Deserialize bytes stored in TFRecordFile."""
  feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
  }
  example = tf.io.parse_example(serialized_example, feature_map)
  sequence = tf.io.decode_raw(example['sequence'], tf.bool)
  sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
  sequence = tf.cast(sequence, tf.float32)

  target = tf.io.decode_raw(example['target'], tf.float16)
  target = tf.reshape(target,
                      (metadata['target_length'], metadata['num_targets']))
  target = tf.cast(target, tf.float32)

  return {'sequence': sequence,
          'target': target}

def get_targets(organism):
  targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
  return pd.read_csv(targets_txt, sep='\t')
df_targets_human = get_targets('human')
# df_targets_human.head()

human_dataset = get_dataset('human', 'train')

2024-07-31 19:51:37.974065: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
print(next(iter(human_dataset)))

2024-07-31 19:51:43.387110: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


{'sequence': <tf.Tensor: shape=(131072, 4), dtype=float32, numpy=
array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)>, 'target': <tf.Tensor: shape=(896, 5313), dtype=float32, numpy=
array([[0.10839844, 0.10760498, 0.04425049, ..., 0.        , 0.        ,
        0.        ],
       [0.10162354, 0.09332275, 0.0094986 , ..., 0.        , 0.        ,
        0.        ],
       [0.10272217, 0.15600586, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.07714844, 0.07678223, 0.03509521, ..., 0.        , 0.01934814,
        0.        ],
       [0.07666016, 0.03826904, 0.05648804, ..., 0.        , 0.        ,
        0.        ],
       [0.08319092, 0.06051636, 0.02156067, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>}


In [ ]:
#oh thank god, now we have to figure out how to index into this, but we're right there!!!

# figuring out how to change the order

In [1]:
import json
import functools
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import numpy as np
import h5py
import tensorflow as tf
# import sonnet as snt
from IPython.display import clear_output
import time
import sys
sys.path.append('/data/leslie/sarthak/data/enformer')
# import enformer


def organism_path(organism):
  return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism) #just leads to the organism path, quite easy


def get_dataset(organism, subset, num_threads=1): #this gets the data and deserializes it
  metadata = get_metadata(organism)
  dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset


def get_metadata(organism):
  # Keys:
  # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
  # pool_width, crop_bp, target_length
  path = os.path.join(organism_path(organism), 'statistics.json')
  with tf.io.gfile.GFile(path, 'r') as f:
    return json.load(f)


def tfrecord_files(organism, subset): #this is the function that gets the TFR files, gets all of them and sorts them
  # Sort the values by int(*).
  return sorted(tf.io.gfile.glob(os.path.join(
      organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
  )), key=lambda x: int(x.split('-')[-1].split('.')[0]))


def deserialize(serialized_example, metadata):
  """Deserialize bytes stored in TFRecordFile."""
  feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
  }
  example = tf.io.parse_example(serialized_example, feature_map)
  sequence = tf.io.decode_raw(example['sequence'], tf.bool)
  sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
  sequence = tf.cast(sequence, tf.float32)

  target = tf.io.decode_raw(example['target'], tf.float16)
  target = tf.reshape(target,
                      (metadata['target_length'], metadata['num_targets']))
  target = tf.cast(target, tf.float32)

  return {'sequence': sequence,
          'target': target}

def get_targets(organism):
  targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
  return pd.read_csv(targets_txt, sep='\t')
df_targets_human = get_targets('human')
# df_targets_human.head()

human_dataset = get_dataset('human', 'train')

2024-08-01 13:09:39.620328: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-01 13:09:39.622783: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 13:09:41.031098: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 13:09:41.471741: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 13:09:41.848664: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
#what if I just want to skip 300 elements, how long does that take
from tqdm import tqdm
for i, data in enumerate(tqdm(human_dataset)):
    if i == 300:
        break
#roughly 17 it/s... so on the slower side


0it [00:00, ?it/s]2024-08-01 13:10:27.371224: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
300it [00:18, 16.27it/s]


In [3]:
#let's print the first element
print(next(iter(human_dataset)))

{'sequence': <tf.Tensor: shape=(131072, 4), dtype=float32, numpy=
array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)>, 'target': <tf.Tensor: shape=(896, 5313), dtype=float32, numpy=
array([[0.10839844, 0.10760498, 0.04425049, ..., 0.        , 0.        ,
        0.        ],
       [0.10162354, 0.09332275, 0.0094986 , ..., 0.        , 0.        ,
        0.        ],
       [0.10272217, 0.15600586, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.07714844, 0.07678223, 0.03509521, ..., 0.        , 0.01934814,
        0.        ],
       [0.07666016, 0.03826904, 0.05648804, ..., 0.        , 0.        ,
        0.        ],
       [0.08319092, 0.06051636, 0.02156067, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>}


In [4]:
#let's now create a custom function that can shuffle the order of the dataset
def get_dataset(organism, subset, num_threads=1): #this gets the data and deserializes it
  metadata = get_metadata(organism)
  dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset)[0],
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset

dataset2 = get_dataset('human', 'train')
#print the first element
print(next(iter(dataset2)))

{'sequence': <tf.Tensor: shape=(131072, 4), dtype=float32, numpy=
array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)>, 'target': <tf.Tensor: shape=(896, 5313), dtype=float32, numpy=
array([[0.10839844, 0.10760498, 0.04425049, ..., 0.        , 0.        ,
        0.        ],
       [0.10162354, 0.09332275, 0.0094986 , ..., 0.        , 0.        ,
        0.        ],
       [0.10272217, 0.15600586, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.07714844, 0.07678223, 0.03509521, ..., 0.        , 0.01934814,
        0.        ],
       [0.07666016, 0.03826904, 0.05648804, ..., 0.        , 0.        ,
        0.        ],
       [0.08319092, 0.06051636, 0.02156067, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>}


In [5]:
#iterate through it
for i, data in enumerate(tqdm(dataset2)):
    continue
#roughly 256 elements in the first file. so 256 elements per file is fine

256it [00:15, 16.37it/s]2024-08-01 13:14:19.811734: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
256it [00:15, 16.50it/s]


In [6]:
#now let's swap the order

def get_dataset(organism, subset, num_threads=1): #this gets the data and deserializes it
  metadata = get_metadata(organism)
  files = tfrecord_files(organism, subset)
  #swap elements 0 and 1
  files[0], files[1] = files[1], files[0]
  dataset = tf.data.TFRecordDataset(files,
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset
dataset3 = get_dataset('human', 'train')

In [7]:
print(next(iter(dataset3)))

{'sequence': <tf.Tensor: shape=(131072, 4), dtype=float32, numpy=
array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]], dtype=float32)>, 'target': <tf.Tensor: shape=(896, 5313), dtype=float32, numpy=
array([[0.07519531, 0.01112366, 0.01609802, ..., 0.        , 0.        ,
        0.        ],
       [0.0881958 , 0.01392365, 0.03829956, ..., 0.        , 0.        ,
        0.        ],
       [0.05108643, 0.01622009, 0.01737976, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00142574, 0.001544  , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00675964, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.05609131, 0.0559082 , 0.06762695, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>}


In [8]:
#and now find element 257
for i, data in enumerate(tqdm(dataset3)):
    if i == 256:
        print(data)
        break

256it [00:15, 16.38it/s]

{'sequence': <tf.Tensor: shape=(131072, 4), dtype=float32, numpy=
array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)>, 'target': <tf.Tensor: shape=(896, 5313), dtype=float32, numpy=
array([[0.10839844, 0.10760498, 0.04425049, ..., 0.        , 0.        ,
        0.        ],
       [0.10162354, 0.09332275, 0.0094986 , ..., 0.        , 0.        ,
        0.        ],
       [0.10272217, 0.15600586, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.07714844, 0.07678223, 0.03509521, ..., 0.        , 0.01934814,
        0.        ],
       [0.07666016, 0.03826904, 0.05648804, ..., 0.        , 0.        ,
        0.        ],
       [0.08319092, 0.06051636, 0.02156067, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>}


In [ ]:
#this is just for label, for sequence, should be quite easy to pair it
#we simply have to know the order of shuffled, then can access the specific index easily!
#still have to shuffle it, which is based on the seed, then we just ignore the index that's asked for by the data loader?
#and instead use the next element of the dataset? that could work... but terrible approach...

In [9]:
#let's compare this to the h5py method
import h5py
split = 'train'
data_path=f'/data/leslie/sarthak/data/enformer/data/{split}_seq.npz'
labels = h5py.File(data_path.replace('_seq.npz', '_label.h5'),'r')['labels']
print(labels.shape)

(34021, 896, 5313)


In [10]:
labels[10] #yeah see this is just incredibly slow to the point where it's the slowdown during training! Like 13 seconds

array([[0.09265137, 0.19274902, 0.265625  , ..., 0.        , 0.        ,
        0.        ],
       [0.12792969, 0.16992188, 0.4387207 , ..., 0.        , 0.        ,
        0.        ],
       [0.12878418, 0.14343262, 0.44189453, ..., 0.        , 0.        ,
        0.12255859],
       ...,
       [0.06341553, 0.08654785, 0.27294922, ..., 0.        , 0.        ,
        0.        ],
       [0.15966797, 0.12335205, 0.45141602, ..., 0.9736328 , 0.        ,
        0.        ],
       [0.14868164, 0.10693359, 0.27368164, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [11]:
#let's try this again
labels[0] #should correspond as well!

array([[0.10839844, 0.10760498, 0.04425049, ..., 0.        , 0.        ,
        0.        ],
       [0.10162354, 0.09332275, 0.0094986 , ..., 0.        , 0.        ,
        0.        ],
       [0.10272217, 0.15600586, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.07714844, 0.07678223, 0.03509521, ..., 0.        , 0.01934814,
        0.        ],
       [0.07666016, 0.03826904, 0.05648804, ..., 0.        , 0.        ,
        0.        ],
       [0.08319092, 0.06051636, 0.02156067, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [12]:
#so use dataset 3 and do .skip(256) and then grab the element
dataset3 = get_dataset('human', 'train')
out = dataset3.skip(256).take(1)
print(out)

<_TakeDataset element_spec={'sequence': TensorSpec(shape=(131072, 4), dtype=tf.float32, name=None), 'target': TensorSpec(shape=(896, 5313), dtype=tf.float32, name=None)}>


In [15]:
#if this can be faster, we could potentially not iterate through a tfr file and instead grab the correct element... this could be really smart!!

for elem in out:
    print(elem)
#this ends up being a bit slower, because up to 15 seconds to iterate through...

{'sequence': <tf.Tensor: shape=(131072, 4), dtype=float32, numpy=
array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)>, 'target': <tf.Tensor: shape=(896, 5313), dtype=float32, numpy=
array([[0.10839844, 0.10760498, 0.04425049, ..., 0.        , 0.        ,
        0.        ],
       [0.10162354, 0.09332275, 0.0094986 , ..., 0.        , 0.        ,
        0.        ],
       [0.10272217, 0.15600586, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.07714844, 0.07678223, 0.03509521, ..., 0.        , 0.01934814,
        0.        ],
       [0.07666016, 0.03826904, 0.05648804, ..., 0.        , 0.        ,
        0.        ],
       [0.08319092, 0.06051636, 0.02156067, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>}


2024-08-01 13:25:30.793754: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
#the easy option and the less pythonic one is to just use the tfr and iterate through, can be smart and choose a specific element, however, still quite slow, roughly 8 seconds on average to load it, might even be slower than h5py in some circumstances...
#the other weirder way is to use the dataset object itself, that way will be incredibly quick, can still easily find corresponding sequence because just use seed to shuffle it
#let's see hwo dataloader is called

# zarr

In [3]:
#an alternative package, let's try this with 

batch_size = 128
num_threads = 4
#we're gonna use HDF5! It's the best way to store large numpy arrays and we can store it paired, but we'll store the seq separately
#this will convert both the label and the sequence to the proper format and save it!

import glob
import json
import functools
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import numpy as np
import h5py
import zarr

def organism_path(organism):
  return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism) #just leads to the organism path, quite easy


def get_dataset(organism, subset, num_threads=8): #this gets the data and deserializes it
  metadata = get_metadata(organism)
  dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset


def get_metadata(organism):
  # Keys:
  # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
  # pool_width, crop_bp, target_length
  path = os.path.join(organism_path(organism), 'statistics.json')
  with tf.io.gfile.GFile(path, 'r') as f:
    return json.load(f)


def tfrecord_files(organism, subset): #this is the function that gets the TFR files, gets all of them and sorts them
  # Sort the values by int(*).
  return sorted(tf.io.gfile.glob(os.path.join(
      organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
  )), key=lambda x: int(x.split('-')[-1].split('.')[0]))


def deserialize(serialized_example, metadata):
  """Deserialize bytes stored in TFRecordFile."""
  feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
  }
  example = tf.io.parse_example(serialized_example, feature_map)
  sequence = tf.io.decode_raw(example['sequence'], tf.bool)
  sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
  sequence = tf.cast(sequence, tf.float32)

  target = tf.io.decode_raw(example['target'], tf.float16)
  target = tf.reshape(target,
                      (metadata['target_length'], metadata['num_targets']))
  target = tf.cast(target, tf.float32)

  return {'sequence': sequence,
          'target': target}

def get_targets(organism):
  targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
  return pd.read_csv(targets_txt, sep='\t')


# Function to create and save data incrementally
def create_hdf5_file_and_np(dataset, filename, num_elems, chunk_size, compression='zlib'):
    """
    Convert a TensorFlow dataset to an HDF5 file for the targets, and a tokenized np array for seq

    Args:
        dataset: TensorFlow dataset to be converted.
        filename: Name of the output HDF5 file.
        num_elems: Number of elements in the dataset.
        batch_size: Batch size for processing the dataset.
        chunk_size: Chunk size for HDF5 dataset.
        compression: Compression type for HDF5 dataset.
    """
    zarr_file = zarr.open(filename, mode='w')
    
    float16_data = zarr_file.create_dataset(
        'labels',
        shape=(num_elems, 896, 5313),
        chunks=(chunk_size, 896, 5313),
        dtype='f2',  # Use float32 for Zarr
        compressor=zarr.Blosc(cname=compression, clevel=6, shuffle=zarr.Blosc.BITSHUFFLE),
    )
    idx = 0
    for i, batch in tqdm(enumerate(dataset), total=num_elems // chunk_size):
        label = batch['target'].numpy()
        true_bsize = label.shape[0]
        float16_data[idx:idx + true_bsize] = label
        idx += true_bsize
        


In [4]:
batch_size = 16
human_test = get_dataset('human', 'test', num_threads=num_threads).batch(batch_size)
filename = '/data/leslie/sarthak/data/enformer/data/test_label.zarr'
create_hdf5_file_and_np(human_test, filename, 1937, batch_size)

  0%|          | 0/121 [00:00<?, ?it/s]

122it [03:21,  1.26s/it]                         2024-08-01 13:59:14.920243: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
122it [03:21,  1.65s/it]


In [5]:
labels_zarr = zarr.open(filename, mode='r')['labels']

In [6]:
print(labels_zarr.shape)

(1937, 896, 5313)


In [7]:
#now loop through and access all elements
for elem in tqdm(labels_zarr):
    continue
#eyo, 60 elements/second?? that's very fast!!

100%|██████████| 1937/1937 [00:30<00:00, 63.10it/s]


In [8]:
#try with the h5py
filename = '/data/leslie/sarthak/data/enformer/data/test_label.h5'
import h5py
labels_h5 = h5py.File(filename, 'r')['labels']
print(labels_h5.shape)

(1937, 896, 5313)


In [9]:
for elem in tqdm(labels_h5):
    continue
#lmao 1 element per 8 seconds, that's the speed difference I guess

  0%|          | 4/1937 [00:40<5:25:26, 10.10s/it]


KeyboardInterrupt: 

In [11]:
#the final thing we want to do is save out the data in 32 bit and maybe that's better?
#first close the zarr file using zarr.close

AttributeError: module 'zarr' has no attribute 'close'

In [12]:
def create_hdf5_file_and_np(dataset, filename, num_elems, chunk_size, compression='zlib'):
    """
    Convert a TensorFlow dataset to an HDF5 file for the targets, and a tokenized np array for seq

    Args:
        dataset: TensorFlow dataset to be converted.
        filename: Name of the output HDF5 file.
        num_elems: Number of elements in the dataset.
        batch_size: Batch size for processing the dataset.
        chunk_size: Chunk size for HDF5 dataset.
        compression: Compression type for HDF5 dataset.
    """
    zarr_file = zarr.open(filename, mode='w')
    
    float16_data = zarr_file.create_dataset(
        'labels',
        shape=(num_elems, 896, 5313),
        chunks=(chunk_size, 896, 5313),
        dtype='f4',  # Use float32 for Zarr
        compressor=zarr.Blosc(cname=compression, clevel=9, shuffle=zarr.Blosc.BITSHUFFLE),
    )
    idx = 0
    for i, batch in tqdm(enumerate(dataset), total=num_elems // chunk_size):
        label = batch['target'].numpy()
        true_bsize = label.shape[0]
        float16_data[idx:idx + true_bsize] = label
        idx += true_bsize

In [13]:
batch_size = 16
human_test = get_dataset('human', 'test', num_threads=num_threads).batch(batch_size)
filename = '/data/leslie/sarthak/data/enformer/data/test_label.zarr'
create_hdf5_file_and_np(human_test, filename, 1937, batch_size)

122it [06:18,  2.35s/it]                         2024-08-01 14:10:36.224076: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
122it [06:18,  3.10s/it]


In [14]:
labels_zarr = zarr.open(filename, mode='r')['labels']
print(labels_zarr.shape)
#now loop through and access all elements
for elem in tqdm(labels_zarr):
    continue
#eyo, 60 elements/second?? that's very fast!!
#still extremely fast! 50 it/s!!!

(1937, 896, 5313)


100%|██████████| 1937/1937 [00:37<00:00, 51.21it/s]


In [16]:
#let's access a ranodm element
print(labels_zarr[10])
print(labels_h5[10])

[[0.03237915 0.07928467 0.02059937 ... 0.         0.         0.        ]
 [0.02607727 0.02429199 0.00772476 ... 0.         0.         0.        ]
 [0.04672241 0.0397644  0.02059937 ... 0.         0.         0.        ]
 ...
 [0.0484314  0.04342651 0.02076721 ... 0.         0.         0.        ]
 [0.02223206 0.01392365 0.04684448 ... 0.         0.4777832  0.47851562]
 [0.03710938 0.02336121 0.02510071 ... 0.         0.09411621 0.32055664]]
[[0.03237915 0.07928467 0.02059937 ... 0.         0.         0.        ]
 [0.02607727 0.02429199 0.00772476 ... 0.         0.         0.        ]
 [0.04672241 0.0397644  0.02059937 ... 0.         0.         0.        ]
 ...
 [0.0484314  0.04342651 0.02076721 ... 0.         0.         0.        ]
 [0.02223206 0.01392365 0.04684448 ... 0.         0.4777832  0.47851562]
 [0.03710938 0.02336121 0.02510071 ... 0.         0.09411621 0.32055664]]


In [18]:
#pick 20 random elements and check if all close
import random
for i in tqdm(range(20)):
    idx = random.randint(0, 1937)
    assert np.allclose(labels_zarr[idx], labels_h5[idx])

100%|██████████| 20/20 [02:50<00:00,  8.54s/it]


# comparing the zarr to h5py

In [1]:
import h5py
import zarr
import numpy as np

#load in the h5py file
filename = '/data/leslie/sarthak/data/enformer/data/test_label.h5'
labels_h5 = h5py.File(filename, 'r')['labels']

#now the zarr file
filename = '/data/leslie/sarthak/data/enformer/data/test_label.zarr'
labels_zarr = zarr.open(filename, mode='r')['labels']

print(labels_h5.shape)
print(labels_zarr.shape)

(1937, 896, 5313)
(1937, 896, 5313)


In [2]:
#now grab a random element
idx = 10
print(labels_h5[idx])
print(labels_zarr[idx])

[[0.03237915 0.07928467 0.02059937 ... 0.         0.         0.        ]
 [0.02607727 0.02429199 0.00772476 ... 0.         0.         0.        ]
 [0.04672241 0.0397644  0.02059937 ... 0.         0.         0.        ]
 ...
 [0.0484314  0.04342651 0.02076721 ... 0.         0.         0.        ]
 [0.02223206 0.01392365 0.04684448 ... 0.         0.4777832  0.47851562]
 [0.03710938 0.02336121 0.02510071 ... 0.         0.09411621 0.32055664]]
[[0.03237915 0.07928467 0.02059937 ... 0.         0.         0.        ]
 [0.02607727 0.02429199 0.00772476 ... 0.         0.         0.        ]
 [0.04672241 0.0397644  0.02059937 ... 0.         0.         0.        ]
 ...
 [0.0484314  0.04342651 0.02076721 ... 0.         0.         0.        ]
 [0.02223206 0.01392365 0.04684448 ... 0.         0.4777832  0.47851562]
 [0.03710938 0.02336121 0.02510071 ... 0.         0.09411621 0.32055664]]


In [3]:
#see the size in nbytes of each
print(labels_h5.nbytes/1e9)

36.883951104


In [4]:
from tqdm import tqdm
for i in tqdm(range(labels_h5.shape[0])):
    #load in the value
    h5_val = labels_h5[i]
    zarr_val = labels_zarr[i]
    #check if they're the same
    assert np.allclose(h5_val, zarr_val)

  0%|          | 9/1937 [01:27<5:11:43,  9.70s/it]


KeyboardInterrupt: 

In [5]:
#let's check the train vals

filename = '/data/leslie/sarthak/data/enformer/data/train_label.h5'
labels_h5 = h5py.File(filename, 'r')['labels']

#now the zarr file
filename = '/data/leslie/sarthak/data/enformer/data/train_label.zarr'
labels_zarr = zarr.open(filename, mode='r')['labels']

print(labels_h5.shape)
print(labels_zarr.shape)

(34021, 896, 5313)
(34021, 896, 5313)


In [8]:
for i in tqdm(range(1000)):
    test = labels_zarr[i]
#this one is significantly slower than the test

  4%|▍         | 44/1000 [00:16<05:55,  2.69it/s]


KeyboardInterrupt: 

In [9]:
filename = '/data/leslie/sarthak/data/enformer/data/test_label.zarr'
test_zarr = zarr.open(filename, mode='r')['labels']
for i in tqdm(range(1000)):
    test = test_zarr[i]
    
#also much slower, this is likely due to it being this node has slower transfer speeds, but regardless it's like a 16x speedup

  5%|▍         | 49/1000 [00:17<05:47,  2.74it/s]


KeyboardInterrupt: 

In [10]:
for i in tqdm(range(labels_h5.shape[0])):
    #load in the value
    h5_val = labels_h5[i]
    zarr_val = labels_zarr[i]
    #check if they're the same
    assert np.allclose(h5_val, zarr_val)

  1%|          | 306/34021 [40:00<73:28:05,  7.84s/it]


KeyboardInterrupt: 

# comparing to the dataset class

In [2]:
# final test is compar it to the dataset class and see if it aligns there too!!
#first let's see the zarr speed
import zarr
from tqdm import tqdm
filename = '/data/leslie/sarthak/data/enformer/data/test_label.zarr'
test_zarr = zarr.open(filename, mode='r')['labels']
for i in tqdm(range(1000)):
    test = test_zarr[i]
#this one is twice as fast, that's fine!

  5%|▍         | 47/1000 [00:13<04:26,  3.58it/s]


KeyboardInterrupt: 

In [3]:
import glob
import json
import functools
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import numpy as np
# import h5py
import zarr

def organism_path(organism):
  return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism) #just leads to the organism path, quite easy


def get_dataset(organism, subset, num_threads=8): #this gets the data and deserializes it
  metadata = get_metadata(organism)
  dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset


def get_metadata(organism):
  # Keys:
  # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
  # pool_width, crop_bp, target_length
  path = os.path.join(organism_path(organism), 'statistics.json')
  with tf.io.gfile.GFile(path, 'r') as f:
    return json.load(f)


def tfrecord_files(organism, subset): #this is the function that gets the TFR files, gets all of them and sorts them
  # Sort the values by int(*).
  return sorted(tf.io.gfile.glob(os.path.join(
      organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
  )), key=lambda x: int(x.split('-')[-1].split('.')[0]))


def deserialize(serialized_example, metadata):
  """Deserialize bytes stored in TFRecordFile."""
  feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
  }
  example = tf.io.parse_example(serialized_example, feature_map)
  sequence = tf.io.decode_raw(example['sequence'], tf.bool)
  sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
  sequence = tf.cast(sequence, tf.float32)

  target = tf.io.decode_raw(example['target'], tf.float16)
  target = tf.reshape(target,
                      (metadata['target_length'], metadata['num_targets']))
  target = tf.cast(target, tf.float32)

  return {'sequence': sequence,
          'target': target}

def get_targets(organism):
  targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
  return pd.read_csv(targets_txt, sep='\t')

dataset = get_dataset('human', 'test')

2024-08-02 11:08:33.732266: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-02 11:08:33.740712: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-02 11:08:37.363545: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-02 11:08:38.055862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 11:08:39.179967: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [5]:
it = iter(dataset)
print(next(it))

{'sequence': <tf.Tensor: shape=(131072, 4), dtype=float32, numpy=
array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)>, 'target': <tf.Tensor: shape=(896, 5313), dtype=float32, numpy=
array([[0.04208374, 0.04983521, 0.01158905, ..., 0.        , 0.        ,
        0.        ],
       [0.05300903, 0.04168701, 0.04956055, ..., 0.        , 0.        ,
        0.        ],
       [0.05258179, 0.0109787 , 0.01287842, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.09313965, 0.15600586, 0.32202148, ..., 0.        , 0.        ,
        0.        ],
       [0.07489014, 0.08380127, 0.23303223, ..., 0.        , 0.        ,
        0.        ],
       [0.11328125, 0.13024902, 0.16992188, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>}


In [6]:
print(test_zarr[0])

[[0.04208374 0.04983521 0.01158905 ... 0.         0.         0.        ]
 [0.05300903 0.04168701 0.04956055 ... 0.         0.         0.        ]
 [0.05258179 0.0109787  0.01287842 ... 0.         0.         0.        ]
 ...
 [0.09313965 0.15600586 0.32202148 ... 0.         0.         0.        ]
 [0.07489014 0.08380127 0.23303223 ... 0.         0.         0.        ]
 [0.11328125 0.13024902 0.16992188 ... 0.         0.         0.        ]]


In [7]:
it = iter(dataset)
for i in tqdm(range(1000)):
    data = next(it)
    assert np.allclose(data['target'], test_zarr[i])

  0%|          | 4/1000 [00:01<06:50,  2.42it/s]


AssertionError: 

In [8]:
data['target']

<tf.Tensor: shape=(896, 5313), dtype=float32, numpy=
array([[0.03903198, 0.0092926 , 0.1381836 , ..., 0.        , 0.        ,
        0.        ],
       [0.00475693, 0.        , 0.01335907, ..., 0.        , 0.        ,
        0.        ],
       [0.01573181, 0.02508545, 0.01528931, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.04394531, 0.02938843, 0.00434494, ..., 0.27856445, 0.        ,
        0.0430603 ],
       [0.00847626, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0479126 , 0.03167725, 0.04071045, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>

In [10]:
test_zarr[i]

array([[0.03869629, 0.00365257, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03961182, 0.04876709, 0.00450516, ..., 0.        , 0.        ,
        0.        ],
       [0.04922485, 0.08099365, 0.01802063, ..., 0.        , 0.        ,
        0.09619141],
       ...,
       [0.04544067, 0.08990479, 0.07336426, ..., 0.        , 0.        ,
        0.        ],
       [0.05267334, 0.05166626, 0.        , ..., 0.        , 0.        ,
        0.11645508],
       [0.08911133, 0.04251099, 0.06695557, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [11]:
#why isi this one different??
test_zarr.shape

(1937, 896, 5313)

In [12]:
#load in labels h5
filename = '/data/leslie/sarthak/data/enformer/data/test_label.h5'
labels_h5 = h5py.File(filename, 'r')['labels']
labels_h5[i]

array([[0.03869629, 0.00365257, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03961182, 0.04876709, 0.00450516, ..., 0.        , 0.        ,
        0.        ],
       [0.04922485, 0.08099365, 0.01802063, ..., 0.        , 0.        ,
        0.09619141],
       ...,
       [0.04544067, 0.08990479, 0.07336426, ..., 0.        , 0.        ,
        0.        ],
       [0.05267334, 0.05166626, 0.        , ..., 0.        , 0.        ,
        0.11645508],
       [0.08911133, 0.04251099, 0.06695557, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [13]:
#check tfrecord files
tfrecord_files('human', 'test')

['/data/leslie/sarthak/data/enformer/data/human/tfrecords/test-0-0.tfr',
 '/data/leslie/sarthak/data/enformer/data/human/tfrecords/test-0-1.tfr',
 '/data/leslie/sarthak/data/enformer/data/human/tfrecords/test-0-2.tfr',
 '/data/leslie/sarthak/data/enformer/data/human/tfrecords/test-0-3.tfr',
 '/data/leslie/sarthak/data/enformer/data/human/tfrecords/test-0-4.tfr',
 '/data/leslie/sarthak/data/enformer/data/human/tfrecords/test-0-5.tfr',
 '/data/leslie/sarthak/data/enformer/data/human/tfrecords/test-0-6.tfr',
 '/data/leslie/sarthak/data/enformer/data/human/tfrecords/test-0-7.tfr']

In [18]:
dataset = get_dataset('human', 'test', num_threads=4)
it = iter(dataset)
for i in tqdm(range(1000)):
    data = next(it)
    assert np.allclose(data['target'], test_zarr[i])



































  3%|▎         | 33/1000 [00:10<05:18,  3.04it/s]


KeyboardInterrupt: 

In [22]:
#so it appears that the threads is a real concern
#thankfully it appears it's almost identical between the way it's loaded, but i don't know...
seq = np.load('/data/leslie/sarthak/data/enformer/data/test_sequence.npy')
seq.shape

(1937, 131072)

In [26]:
seq = np.load('/data/leslie/sarthak/data/enformer/data/test_seq.npz')
seq['sequence_array']

array([[10,  7, 10, ...,  9,  7,  9],
       [10,  7,  7, ..., 10,  7, 10],
       [10, 10,  8, ...,  7,  7,  9],
       ...,
       [10,  9,  7, ...,  8, 10, 10],
       [10,  8,  7, ...,  9,  8,  8],
       [ 7,  8,  8, ..., 10, 10,  9]], dtype=int8)

In [29]:
a = next(iter(dataset))['sequence']
print(a.shape)
a

(131072, 4)


<tf.Tensor: shape=(131072, 4), dtype=float32, numpy=
array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)>

# testing sequence

In [31]:
#let's check the first sequence and if it matches this

#chr10	37555537	37686609	test

import pyfaidx
# ref genome is located /data/leslie/sarthak/data/chrombpnet_test/hg38.fa
# let's extract the sequence for chr10:37555537-37686609
fasta = pyfaidx.Fasta('/data/leslie/sarthak/data/chrombpnet_test/hg38.fa')
seq = fasta['chr10'][37555537:37686609].seq
seq

'...'

In [33]:
#now let's convert a to a string
def convert_to_string(a):
    #convert to string
    seq = ''
    for i in tqdm(range(a.shape[0])):
        if a[i, 0] == 1:
            seq += 'A'
        elif a[i, 1] == 1:
            seq += 'C'
        elif a[i, 2] == 1:
            seq += 'G'
        elif a[i, 3] == 1:
            seq += 'T'
    return seq
stringseq = convert_to_string(a)
stringseq

100%|██████████| 131072/131072 [01:24<00:00, 1556.77it/s]


'...'

In [34]:
#looks identical!
seq == stringseq
#ok, so the bed file is in order

True

In [47]:
#let's test it by looking at example 603
#chrX	132168903	132299975	test
dataset = get_dataset('human', 'test', num_threads=1) #make sure num threads is 1!
it = iter(dataset)
for i in tqdm(range(603)):
    data = next(it)
# data = next(it) #this should be element 603

100%|██████████| 603/603 [00:33<00:00, 17.98it/s]


In [48]:
data

{'sequence': <tf.Tensor: shape=(131072, 4), dtype=float32, numpy=
 array([[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        ...,
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]], dtype=float32)>,
 'target': <tf.Tensor: shape=(896, 5313), dtype=float32, numpy=
 array([[0.01841736, 0.0067215 , 0.00482941, ..., 0.        , 0.        ,
         0.        ],
        [0.00119877, 0.0221405 , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.00652313, 0.01207733, 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.0617981 , 0.01374817, 0.02703857, ..., 0.        , 0.        ,
         0.        ],
        [0.02204895, 0.00998688, 0.01577759, ..., 0.        , 0.        ,
         0.        ],
        [0.02874756, 0.02783203, 0.01255035, ..., 0.        , 0.        ,
         0.        ]], dtype=float32)>}

In [49]:
test_zarr[603] #this one is actually 604!

array([[8.5083008e-02, 2.9891968e-02, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 4.8291016e-01],
       [6.6162109e-02, 1.9317627e-02, 1.6093254e-04, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [7.3425293e-02, 3.1372070e-02, 2.8488159e-02, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [6.0424805e-02, 5.1696777e-02, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [5.7891846e-02, 3.4515381e-02, 2.7359009e-02, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.0424805e-01, 4.7882080e-02, 2.0919800e-02, ..., 0.0000000e+00,
        7.3303223e-02, 0.0000000e+00]], dtype=float32)

In [50]:
stringseq = convert_to_string(data['sequence'])
seq = fasta['chrX'][132168903:132299975].seq

100%|██████████| 131072/131072 [01:24<00:00, 1543.84it/s]


In [51]:
seq==stringseq

True

In [54]:
(196608-131072)//2

32768

In [65]:
#seems good, so we have to redo it, then extend the sequence, save the one hot encoded data, and then convert it to tokenized form
#now the key is we don't want to just use 131072 of the data, but extend to 196608, so let's see how to extend the sequence
start = 132168903; end = 132299975; chrom = 'chrX'; original_length = 131072
#we want to extend to 196608
diff = (196608 - original_length)//2
start = start - diff
end = end + diff
seq = fasta[chrom][start:end].seq
print(len(seq))

196608


In [66]:
diff

32768

In [70]:
ogseq = fasta['chrX'][132168903:132299975].seq
ogseq==seq[32768:-32768]

True

In [68]:
len(ogseq)

131072

In [71]:
len(seq[32768:-32768])

131072

# figuring out how to extend and save seq

In [72]:
genome_np = np.load('/data/leslie/sarthak/data/chrombpnet_test/hg38_tokenized.npz')
seqs = pd.read_csv('/data/leslie/sarthak/data/enformer/data/human/sequences.bed', sep='\t', header=None)
seqs.head()

,0,1,2,3
0,chr18,928386,1059458,train
1,chr4,113630947,113762019,train
2,chr11,18427720,18558792,train
3,chr16,85805681,85936753,train
4,chr3,158386188,158517260,train


In [73]:
#find only the ones in column 3 that are test
test_seqs = seqs[seqs[3] == 'test']
print(test_seqs.shape) #looks good!

(1937, 4)


In [76]:
#now access row 10
row = test_seqs.iloc[10]
row

0       chr14
1    97944300
2    98075372
3        test
Name: 36244, dtype: object

In [77]:
genome_np[chrom][start:end]

array([10,  9,  9, ...,  9,  7,  9], dtype=int8)

In [78]:
genome_np = '/data/leslie/sarthak/data/chrombpnet_test/hg38_tokenized.npz'
with np.load(genome_np) as data:
    genome = {key: np.array(data[key]) for key in data}

In [79]:
genome[chrom][start:end]

array([10,  9,  9, ...,  9,  7,  9], dtype=int8)

In [ ]:
#wait genius idea, what if we just save otu the zarr files? that way we can easily access data from the bed file if we need it!
#we have super fast acess to it, so sequence access via bed, and target access via zarr, that way we can easily access the data!

# first check if we extend are there any problematic regions of the chromosome

In [1]:
chrom_info= {'chr1': [10000, 248946422],
 'chr2': [10000, 242183529],
 'chr3': [10000, 198235559],
 'chr4': [10000, 190204555],
 'chr5': [10000, 181478259],
 'chr6': [60000, 170745979],
 'chr7': [10000, 159335973],
 'chr8': [60000, 145078636],
 'chr9': [10000, 138334717],
 'chr10': [10000, 133787422],
 'chr11': [60000, 135076622],
 'chr12': [10000, 133265309],
 'chr13': [16000000, 114354328],
 'chr14': [16022637, 106883718],
 'chr15': [17000000, 101981189],
 'chr16': [10000, 90228345],
 'chr17': [60000, 83247441],
 'chr18': [10000, 80263285],
 'chr19': [60000, 58607616],
 'chr20': [60000, 64334167],
 'chr21': [5010000, 46699983],
 'chr22': [10510000, 50808468],
 'chrX': [10000, 156030895],
 'chrY': [2781479, 56887902],
}

import pandas as pd
#load in the bed file
seqs = pd.read_csv('/data/leslie/sarthak/data/enformer/data/human/sequences.bed', sep='\t', header=None)
seqs.head()

,0,1,2,3
0,chr18,928386,1059458,train
1,chr4,113630947,113762019,train
2,chr11,18427720,18558792,train
3,chr16,85805681,85936753,train
4,chr3,158386188,158517260,train


In [2]:
#now loop through it
seqs = seqs.to_numpy()

In [7]:
from tqdm import tqdm
for i in tqdm(range(seqs.shape[0])):
    row = seqs[i]
    chrom = row[0]
    start = row[1]
    end = row[2]
    length = end - start
    diff = (196608 - length)//2
    start = start - diff
    end = end + diff
    if start < chrom_info[chrom][0]:
        print(f'Error: {chrom} {start} {end} {row[3]}, iter: {i}')

  0%|          | 0/38171 [00:00<?, ?it/s]

100%|██████████| 38171/38171 [00:00<00:00, 505073.67it/s]

Error: chr3 -22768 173840 train, iter: 1022
Error: chr4 -22768 173840 train, iter: 25870
Error: chr5 -22768 173840 train, iter: 27331
Error: chr16 -22768 173840 train, iter: 28360
Error: chr2 -22768 173840 train, iter: 30217
Error: chr12 -22768 173840 train, iter: 31591
Error: chr20 33567 230175 valid, iter: 34218
Error: chr7 -22768 173840 valid, iter: 35954


In [8]:
from tqdm import tqdm
for i in tqdm(range(seqs.shape[0])):
    row = seqs[i]
    chrom = row[0]
    start = row[1]
    end = row[2]
    length = end - start
    diff = (196608 - length)//2
    start = start - diff
    end = end + diff
    if end > chrom_info[chrom][1]:
        print(f'Error: {chrom} {start} {end} {row[3]}, iter: {i}')

100%|██████████| 38171/38171 [00:00<00:00, 499420.66it/s]

Error: chr21 46509644 46706252 train, iter: 10285
Error: chr7 159149400 159346008 train, iter: 13701
Error: chr17 83078345 83274953 train, iter: 23503
Error: chr11 134909011 135105619 train, iter: 26220
Error: chr20 64159289 64355897 valid, iter: 34094


In [11]:
#so few examples, all in train 3 in valid, 0 in test. For those examples, just pad with N
#if it's not negative, no need to even pad!
import numpy as np
genome_np = '/data/leslie/sarthak/data/chrombpnet_test/hg38_tokenized.npz'
total_bytes = 0
with np.load(genome_np) as data:
    genome = {key: np.array(data[key]) for key in data}
total_bytes += sum([genome[key].nbytes for key in genome])

In [12]:
total_bytes//1e9
#3gb for the whole genome, makes sense as it's 3 billion base pairs each taking 1 byte

3.0

In [13]:
#so if we wanted to store all the sequences, that is 
34021*196608*1/1e9 #yeah, just put it all into memory

6.688800768

In [15]:
a = np.zeros(0)
a

array([], dtype=float64)

In [ ]:
split = 'test'
max_length = 196608
genome_np = '/data/leslie/sarthak/data/chrombpnet_test/hg38_tokenized.npz'
total_bytes = 0
with np.load(genome_np) as data:
    genome = {key: np.array(data[key]) for key in data}

In [23]:
seqs = pd.read_csv('/data/leslie/sarthak/data/enformer/data/human/sequences.bed', sep='\t', header=None)
seqs_bed = seqs[seqs[3] == split]
seqs = np.zeros((len(seqs_bed), max_length), dtype=np.int8)
length = 131072
seqs_np = seqs_bed.to_numpy()
for i in range(seqs_np.shape[0]):
    row = seqs_np[i]
    chrom = row[0]
    start = row[1]
    end = row[2]
    diff = (196608 - length)//2 #note this works even if we want a shorter length!
    start = start - diff
    end = end + diff
    leftpad = np.zeros(0)
    rightpad = np.zeros(0)
    if start < 0:
        leftpad = np.ones(-start)*11
        start = 0
    chromlen = chrom_info[chrom][1]
    if end > chromlen:
        rightpad = np.ones(end-chromlen)*11
        end = chromlen
    seq = np.concatenate([leftpad, genome[chrom][start:end], rightpad])
    seqs[i] = seq

In [24]:
seqs

array([[10, 10,  9, ...,  8, 10, 10],
       [10,  7,  8, ...,  8,  8,  8],
       [ 7, 10,  9, ...,  8,  8, 10],
       ...,
       [10,  7,  7, ..., 10,  8, 10],
       [ 7,  9,  9, ..., 10,  8,  8],
       [ 8, 10,  8, ...,  7,  7,  7]], dtype=int8)

In [25]:
seqs.shape

(1937, 196608)

In [26]:
#let's try val which has values that have issues
split = 'valid'
seqs = pd.read_csv('/data/leslie/sarthak/data/enformer/data/human/sequences.bed', sep='\t', header=None)
seqs_bed = seqs[seqs[3] == split]
seqs = np.zeros((len(seqs_bed), max_length), dtype=np.int8)
length = 131072
seqs_np = seqs_bed.to_numpy()
for i in range(seqs_np.shape[0]):
    row = seqs_np[i]
    chrom = row[0]
    start = row[1]
    end = row[2]
    diff = (196608 - length)//2 #note this works even if we want a shorter length!
    start = start - diff
    end = end + diff
    leftpad = np.zeros(0)
    rightpad = np.zeros(0)
    if start < 0:
        leftpad = np.ones(-start)*11
        start = 0
    chromlen = chrom_info[chrom][1]
    if end > chromlen:
        rightpad = np.ones(end-chromlen)*11
        end = chromlen
    seq = np.concatenate([leftpad, genome[chrom][start:end], rightpad])
    seqs[i] = seq

In [27]:
seqs

array([[ 8,  8, 10, ...,  7, 10, 10],
       [ 7,  9,  9, ..., 10, 10, 10],
       [ 7, 10,  7, ...,  8,  7,  8],
       ...,
       [ 9,  9,  9, ...,  9,  8,  8],
       [10, 10,  8, ...,  8,  9, 10],
       [10, 10,  9, ..., 10,  9, 10]], dtype=int8)

In [29]:
seqs[34094-34021][-100:] #see it's padded with 11!

array([11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11],
      dtype=int8)

In [31]:
#this works, now load the corresponding data from the zarr file
import zarr
split = 'test'
filename = f'/data/leslie/sarthak/data/enformer/data/{split}_label.zarr'
labels_zarr = zarr.open(filename, mode='r')['labels']
print(labels_zarr.shape)

(1937, 896, 5313)


In [32]:
labels_zarr[-1]

array([[0.11419678, 0.06634521, 0.11108398, ..., 0.        , 0.        ,
        0.        ],
       [0.10870361, 0.07550049, 0.09332275, ..., 0.        , 0.        ,
        0.48217773],
       [0.07104492, 0.04669189, 0.08917236, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.13830566, 0.13903809, 0.17224121, ..., 0.        , 0.        ,
        0.        ],
       [0.11737061, 0.0871582 , 0.03991699, ..., 0.        , 0.        ,
        0.        ],
       [0.08679199, 0.08294678, 0.02993774, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [34]:
for i in tqdm(range(1000)):
    test = labels_zarr[i]
#this is on the slower side but not bad at all
#potentially it's slower because we hve lowercompression?
#will test that too

  5%|▍         | 49/1000 [00:33<10:51,  1.46it/s]


KeyboardInterrupt: 

# lets create a faster tokenizer too

In [37]:
#if we want 8 elements use itertools
import itertools
numbers = (7, 8, 9, 10, 11)
length = 6

# Generate all combinations of length 8
combinations = itertools.product(numbers, repeat=length)

# Create a dictionary mapping each combination to a number
combination_dict = {comb: idx for idx, comb in enumerate(combinations)}

# Verify the dictionary size
print(f"Number of combinations: {len(combination_dict)}")  # Output: 65536

# Example: Print a few entries to verify
for i, (key, value) in enumerate(combination_dict.items()):
    print(f"{key}: {value}")
    if i == 10:  # Only print the first 5 for brevity
        break

Number of combinations: 15625
(7, 7, 7, 7, 7, 7): 0
(7, 7, 7, 7, 7, 8): 1
(7, 7, 7, 7, 7, 9): 2
(7, 7, 7, 7, 7, 10): 3
(7, 7, 7, 7, 7, 11): 4
(7, 7, 7, 7, 8, 7): 5
(7, 7, 7, 7, 8, 8): 6
(7, 7, 7, 7, 8, 9): 7
(7, 7, 7, 7, 8, 10): 8
(7, 7, 7, 7, 8, 11): 9
(7, 7, 7, 7, 9, 7): 10


In [38]:
len(combination_dict) #exactly what we expect, 5^6 because kmers of length 6!

15625

In [39]:
a = np.array([7,8,7,7,7,9,8,7,10,10,10,7,7,11,11,10])
testtuple = tuple(a[0:6])
testtuple

(7, 8, 7, 7, 7, 9)

In [40]:
combination_dict[testtuple] #ok it found it!

627

In [41]:
combination_dict[(11,11,11,11,11,11)] #the very last element, this is what we will set it to at the right ends so that the lengths are same

15624

In [43]:
genome['chr1'].shape

(248956422,)

In [45]:
#this way we can use int16
kmer_genome = {}
for chrom in genome:
    kmer_genome[chrom] = np.ones((genome[chrom].shape), dtype=np.int16)*15624 #the default option
    for i in tqdm(range(genome[chrom].shape[0]-5)):
        kmer = tuple(genome[chrom][i:i+6])
        kmer_genome[chrom][i] = combination_dict[kmer]
    break

 29%|██▊       | 71325422/248956417 [12:57<32:16, 91725.06it/s]  


KeyboardInterrupt: 

In [46]:
#so let's test the speed of the zarr file
labels_zarr = zarr.open(filename, mode='r')['labels']
filename = f'/data/leslie/sarthak/data/enformer/data/{split}_label_9.zarr'
labels_zarr9 = zarr.open(filename, mode='r')['labels']

print(labels_zarr.shape, labels_zarr9.shape)

(1937, 896, 5313) (1937, 896, 5313)


In [48]:
#now loop through and see the speed
for i in tqdm(range(50)):
    test = labels_zarr[i]

100%|██████████| 50/50 [00:43<00:00,  1.14it/s]


In [49]:
for i in tqdm(range(50)):
    test = labels_zarr9[i]
    
#marginal to no speed icnrease, but it's smaller, so potentially later? We'll just leave it as it is for now

100%|██████████| 50/50 [00:42<00:00,  1.16it/s]


# final test, compare the values

In [1]:
#is it truly similar what i'm doing with what they have

#first thing we do is load in the dataset

import sys
sys.path.append('/data/leslie/sarthak/hyena/hyena-dna/')
import src.dataloaders.datasets.enformer_dataset as enformer_dataset
dataset = enformer_dataset.EnformerDataset('test', 196608, return_CAGE=True)

2024-08-02 23:39:37.430011: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-02 23:39:41.262100: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 23:39:42.180534: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 23:39:42.245579: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-02 23:39:43.558249: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import json
import functools
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import numpy as np
import h5py
import tensorflow as tf
# import sonnet as snt
from IPython.display import clear_output
import time
import sys
sys.path.append('/data/leslie/sarthak/data/enformer')
# import enformer


def organism_path(organism):
  return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism) #just leads to the organism path, quite easy


def get_dataset(organism, subset, num_threads=1): #this gets the data and deserializes it
  metadata = get_metadata(organism)
  dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset


def get_metadata(organism):
  # Keys:
  # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
  # pool_width, crop_bp, target_length
  path = os.path.join(organism_path(organism), 'statistics.json')
  with tf.io.gfile.GFile(path, 'r') as f:
    return json.load(f)


def tfrecord_files(organism, subset): #this is the function that gets the TFR files, gets all of them and sorts them
  # Sort the values by int(*).
  return sorted(tf.io.gfile.glob(os.path.join(
      organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
  )), key=lambda x: int(x.split('-')[-1].split('.')[0]))


def deserialize(serialized_example, metadata):
  """Deserialize bytes stored in TFRecordFile."""
  feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
  }
  example = tf.io.parse_example(serialized_example, feature_map)
  sequence = tf.io.decode_raw(example['sequence'], tf.bool)
  sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
  sequence = tf.cast(sequence, tf.float32)

  target = tf.io.decode_raw(example['target'], tf.float16)
  target = tf.reshape(target,
                      (metadata['target_length'], metadata['num_targets']))
  target = tf.cast(target, tf.float32)

  return {'sequence': sequence,
          'target': target}

def get_targets(organism):
  targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
  return pd.read_csv(targets_txt, sep='\t')
df_targets_human = get_targets('human')
# df_targets_human.head()

dataset_tf = get_dataset('human', 'test', num_threads=1)


2024-08-02 16:50:11.778276: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
dataset.seq = dataset.seqs

In [13]:
import torch
it = iter(dataset_tf)
for i in tqdm(range(1937)):
    data = next(it)
    seq,label = dataset[i]
    assert np.allclose(data['target'], label)
    cutseq = seq[32768:-32768].unsqueeze(0)
    x_onehot = torch.nn.functional.one_hot((cutseq-7)%4, num_classes=4).float().transpose(1, 2) #need to make sure it is the right order
    if 11 in cutseq:
        indices = torch.where(cutseq == 11)
        for idx in range(len(indices[0])):
            x_onehot[indices[0][idx], 0, indices[1][idx]] = 0
    assert np.allclose(data['sequence'], x_onehot.squeeze().transpose(0,1).numpy())

  0%|          | 0/1937 [00:00<?, ?it/s]

100%|██████████| 1937/1937 [27:49<00:00,  1.16it/s] 


In [3]:
#and val data
dataset = enformer_dataset.EnformerDataset('valid', 196608, return_CAGE=True)
dataset_tf = get_dataset('human', 'valid', num_threads=1)

In [ ]:
dataset.seq

array([], shape=(0, 196608), dtype=int8)

In [4]:
import torch
it = iter(dataset_tf)
for i in tqdm(range(2213)):
    data = next(it)
    seq,label = dataset[i]
    assert np.allclose(data['target'], label)
    cutseq = seq[32768:-32768].unsqueeze(0)
    x_onehot = torch.nn.functional.one_hot((cutseq-7)%4, num_classes=4).float().transpose(1, 2) #need to make sure it is the right order
    if 11 in cutseq:
        indices = torch.where(cutseq == 11)
        for idx in range(len(indices[0])):
            x_onehot[indices[0][idx], 0, indices[1][idx]] = 0
    assert np.allclose(data['sequence'], x_onehot.squeeze().transpose(0,1).numpy())

100%|██████████| 2213/2213 [27:39<00:00,  1.33it/s]


In [2]:
#so we can confirm that this is good!
#so let's retrain!!
dataset = enformer_dataset.EnformerDataset('train', 196608, return_CAGE=True)
for i in range(40):
    seq,label = dataset[i]
    print(seq.shape, label.shape)

#loads plenty fast, but slow part is still 

torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size([196608]) torch.Size([896, 5313])
torch.Size